# Introduction

[HHL algorithm](https://en.wikipedia.org/wiki/Quantum_algorithm_for_linear_systems_of_equations) 

これは[Bayesian Deep Learning on a Quantum Computer](https://arxiv.org/abs/1806.11463)という論文の[computational appendix](https://gitlab.com/apozas/bayesian-dl-quantum) の実装を元にしています。

In [0]:
!pip install qiskit==0.8.0
!pip install qiskit-aqua==0.4.0

In [0]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import BasicAer
π = np.pi

# Setting up the problem

$Ax=b$ をときます。$A = \frac{1}{2}\begin{bmatrix}3 & 1 \\1 & 3 \\ \end{bmatrix}$ 、$b =\begin{bmatrix} 1 \\ 0 \\ \end{bmatrix}$とします。

 $A$をハミルトニアン、 $b$をレジスタにする。

補助的には、合計5つのqubitsと1つの古典的レジスタが必要になります。結果を理想的な状態と比較するためのスワップテストを作成するために、追加の量子ビットと追加の古典的なレジスタを追加します。

In [0]:
q = QuantumRegister(6)
c = ClassicalRegister(2)
hhl = QuantumCircuit(q, c)

$b$ は$\left|b\right\rangle = \sum_{i=0}^N b_i\left|i\right\rangle = \left|0\right\rangle$としてエンコードされます。

したがって、この場合、明示的な状態準備回路は必要ありません。

# Quantum phase estimation

 **正直ここはなぜこのコントロールゲートになったのかよくわからなかったので、誰か教えてください笑。コードは論文に添付されているこれ(https://gitlab.com/apozas/bayesian-dl-quantum/blob/master/HHL_IBMQ.py) です。 **
 
 まぁそれだけ発展途上の分野ということで……（お茶を濁す）
 
以下説明の英訳を貼ります。


次のステップは、行列$A$の固有値を追加のレジスタに符号化することです。これは、ある時間$ t_0 $、$ \ exp（i A t_0）$の間にハミルトニアン$ A $によって表される時間発展の量子位相推定によって行われます。プロトコルは3つのステップがあります。


まずアンシラを用意します $\left|\psi_0\right\rangle=\sum_{\tau=0}^{T-1}\left|\tau\right\rangle$。

なぜこの状態か？

これは時間発展を制御します。それは時計のようなもので、一定の時間発展をオンにします。オリジナルのHHLアルゴリズムは、アルゴリズムの次のステップでの誤差を最小にするすべての状態の重み付き重ね合わせを提案しています。しかし、我々の実装では、均一な重ね合わせはすでに良い結果をもたらしています。

私たちの目標は、異なる期間に適用されるハミルトニアンとして$ A $の重ね合わせを作成することです。固有値は常に複素単位円上にあるので、重ね合わせにおいてこれらの異なって進化した成分は固有構造を明らかにするのに役立ちます。だから我々は条件付きハミルトニアン進化を適用する
$\sum_{\tau=0}^{T-1}\left|\tau\right\rangle\left\langle\tau\right|\otimes e^{i A\tau t_0/T}$ on $\left|\psi_0\right\rangle\otimes\left|b\right\rangle$. 

この操作は、時間$ \tau $に対する、状態 $\left|\psi_0\right\rangle$で決定されるハミルトニアン$ A $によって決定される $\left|b\right\rangle$で時間発展します。 

$\left|\psi_0\right\rangle$では、$ 0 $と$ T $の間のすべての可能な時間ステップの重ね合わせがあり、最終的にすべての可能な時間発展の重ね合わせと、適切な数のタイムステップ$ T $と総進化時間$ t_0 $により、固有値のバイナリ表現をエンコードできます。

最後のステップとして、位相を出力する（つまり、$ A $の固有値をエンコードする）逆フーリエ変換を新しいレジスタに適用します。

私たちの$ 2 \ times 2 $の場合、回路は非常に単純化されています。行列$ A $に$ 2 $のべき乗である固有値があると仮定すると、$ T = 4 $、$ t_0 = 2 \ pi $を選択して、2つの制御された展開で正確な結果を得ることができます。

In [0]:
# Superposition
hhl.h(q[1])
hhl.h(q[2])
# Controlled-U0
hhl.cu3(-π / 2, -π / 2, π / 2, q[2], q[3])
hhl.cu1(3 * π / 4, q[2], q[3])
hhl.cx(q[2], q[3])
hhl.cu1(3 * π / 4, q[2], q[3])
hhl.cx(q[2], q[3])
# Controlled-U1
hhl.cx(q[1], q[3]);

位相をレジスタに書き込むために量子逆フーリエ変換を適用します。


In [0]:
hhl.swap(q[1], q[2])
hhl.h(q[2])
hhl.cu1(-π / 2, q[1], q[2])
hhl.h(q[1]);

この分解後のシステムの状態は、次のとおりです。

 $\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle \left|\lambda_{j}\right\rangle$
 
 $\left|b\right\rangle=\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle$は $A$の固有ベクトルでの$b$ベクトルのエンコーディングです。

次に$\left|\lambda_{j}\right\rangle$で実際にそれを反転する。

この場合、固有値の反転は簡単です。
$A$ の固有値は$\lambda_1=2=10_2$ と $\lambda_2=1=01_2$

そしてその逆数は$\lambda_1^{-1}=1/2$ と $\lambda_2^{-1}=1$. 

$2\lambda_1^{-1}=01_2$ と $2\lambda_2^{-1}=10_2$はスワップゲートで$\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle \left|2\lambda _{j}^{-1}\right\rangle$という状態を取得するのに十分です。

In [0]:
hhl.swap(q[1], q[2]);

# Conditional rotation of ancilla

次に、条件付き回転を実行して、固有値の逆数の情報を状態の振幅に符号化します。

取得したい状態は$\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle\left|2\lambda _{j}^{-1}\right\rangle \left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\left|0\right\rangle+\frac{C}{\lambda_j}\left|1\right\rangle \right)$です。


これは、条件付きハミルトニアン発展と精神の中で制御された回転によって達成される。らしい。
**（ここもよくわからない）**

In [0]:
hhl.cu3(0.392699, 0, 0, q[1], q[0])  # Controlled-RY0
hhl.cu3(0.19634955, 0, 0, q[2], q[0]);  # Controlled-RY1

# Uncomputing the eigenvalue register

次に必要なステップは、アルゴリズムから取得したい情報を最終レジスタに格納する操作以外のすべての操作を逆順にすることです。レジスタがエンタングルしている場合はこれを行う必要があります。これは結果に影響します。 

この場合、位相推定プロトコルを計算しないでください。計算が終了した後の状態は$\sum_{j=1}^N\beta_j\left|u_j\right\rangle\left|0\right\rangle\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\left|0\right\rangle+\frac{C}{\lambda_j}\left|1\right\rangle \right)$


In [0]:
hhl.swap(q[1], q[2])
hhl.h(q[1])
hhl.cu1(π / 2, q[1], q[2])  # Inverse(Dagger(Controlled-S))
hhl.h(q[2])
hhl.swap(q[2], q[1])
# Inverse(Controlled-U1)
hhl.cx(q[1], q[3])
# Inverse(Controlled-U0)
hhl.cx(q[2], q[3])
hhl.cu1(-3 * π / 4, q[2], q[3])
hhl.cx(q[2], q[3])
hhl.cu1(-3 * π / 4, q[2], q[3])
hhl.cu3(-π / 2, π / 2, -π / 2, q[2], q[3])
# End of Inverse(Controlled-U0)
hhl.h(q[2])
hhl.h(q[1]);

hhl.measure(q[0], c[0])

# Rejection sampling on the ancilla register and a swap test

状態 $\left|x\right\rangle=A^{-1}\left|b\right\rangle\propto\sum_j \beta_j\lambda_j^{-1}\left|u_j\right\rangle$は $ Ax = b $の解に関する情報を含み、補助状態で$ 1 $を測定したときに得られるものです。目的の$\left|1\right\rangle$に射影して事後選択を実行します。

その解が期待通りのものであることを確認するために、正しい出力状態を手動で準備し、結果を使ってスワップテストを実行します。

In [0]:
# Target state preparation
hhl.rz(-π, q[4])
hhl.u1(π, q[4])
hhl.h(q[4])
hhl.ry(-0.9311623288419387, q[4])
hhl.rz(π, q[4])
# Swap test
hhl.h(q[5])
hhl.cx(q[4], q[3])
hhl.ccx(q[5], q[3], q[4])
hhl.cx(q[4], q[3])
hhl.h(q[5])

hhl.barrier(q)
hhl.measure(q[0], c[0])
hhl.measure(q[5], c[1]);

2つの測定が実行されます。1つは補助的なレジスタ（選択後を実行するため）で、もう1つはスワップテストの結果を示します。

成功確率を計算するために、ヘルパー関数を定義しましょう。

In [0]:
def get_psuccess(counts):
    '''Compute the success probability of the HHL protocol from the statistics

    :return: (float) The success probability.
    '''
    try:
        succ_rotation_fail_swap = counts['11']
    except KeyError:
        succ_rotation_fail_swap = 0
    try:
        succ_rotation_succ_swap = counts['01']
    except KeyError:
        succ_rotation_succ_swap = 0
    succ_rotation = succ_rotation_succ_swap + succ_rotation_fail_swap
    try:
        prob_swap_test_success = succ_rotation_succ_swap / succ_rotation
    except ZeroDivisionError:
        prob_swap_test_success = 0
    return prob_swap_test_success

最後に、シミュレータで回路を実行します。

In [0]:
backend = BasicAer.get_backend('qasm_simulator')
job = execute(hhl, backend, shots=100)
result = job.result()
counts = result.get_counts(hhl)
print(get_psuccess(counts))

1.0


Running on the actual QPU would yield a much poorer result due to imprecisions in the applications of the gates and noise caused by the environment.

# References
[1] J. Pan, Y. Cao, X. Yao, Z. Li, C. Ju, H. Chen, X. Peng, S. Kais, and J. Du. (2014). [Experimental realization of quantum algorithm for solving linear systems of equations](https://arxiv.org/abs/1302.1946). *Physical Review Letters* 89:022313. <a id='1'></a>
